# Project Overview for BioCreative V Task 3b

Tong Shu Li<br>
Created on Tuesday, 2015-07-28<br>
Last updated on 2015-12-08

This notebook provides a project overview for the crowdsourcing-only solution to [task 3b](http://www.biocreative.org/tasks/biocreative-v/track-3-cdr/) of BioCreative V (2015), *Chemical-induced diseases relation extraction*.

### Task Description

Task 3 for BioCreative V is split into two subproblems: A and B.

a) Subtask A focuses on named entity recognition (NER) of chemicals and diseases from raw text. The entities must be identified in free text and mapped to MeSH identifiers.<br>
b) Subtask B focuses on finding relationships where chemicals induce diseases from raw text.

Task 3b:

**Input:** Raw text representing the title and abstract of a scientific publication.<br>
**Output:** A list of MeSH identifier pairs representing chemical-induced disease relationships asserted explicitly in the text. Each relationship should be assigned a normalized confidence score.

### Notes

Since subtask B depends explicitly on the NER step of subtask A, tools are provided to perform NER (tmChem for chemicals and DNorm for diseases).

For this crowdsourcing solution for subtask 3b, no effort is made to perform the NER step *de novo*. Instead, the provided tools (tmChem and DNorm) are used to perform the NER step and the crowd is only asked to identify and extract the chemical-induced disease relationships.

## Solution Workflow

A decision tree diagram representing the overall workflow for the crowdsourcing solution is given below. In brief, all the possible chemical-disease pairs in the text are generated. The pairs are split into three groups: those which follow a clear "[chemical]-induced [disease]" format (CID) within a single sentence, those which do not follow the CID format (but still co-occur within a sentence), and those which do not co-occur within any sentences.

The CID group is automatically always judged to be true. The sentence co-occuring pairs are processed by the crowd in a sentence-level task, and the non-co-occuring pairs are processed with an abstract-level task.

The pairs are split up into different groups to simplify the tasks that workers see. This also reduces costs since sentence-based tasks are easier.

After the tasks are completed by the workers, the final results are aggregated together to produce the CID relations at the abstract level.

<img src="data/notebook/workflow.png">

### Assumptions

There are a number of assumptions with this specific workflow.

1. The NER step is performed perfectly.
2. Some relations following a specific grammatical structure are always true.
3. A sentence contains all the necessary information to judge whether a CID relation is true within that sentence.
4. It is possible to determine whether a CID relation is true at the abstract level solely from judgements about whether that CID relation is true for a subset of the sentences in the abstract.

### Normalized Confidence Score

Each CID relation should be assigned a normalized confidence score. From our previous work with the EU-ADR corpus, we saw that inter-annotator agreement is a good way of identifying ambiguous tasks and problems with a gold standard <sup>[1](http://arxiv.org/abs/1505.06256)</sup>. We also saw that the probability that the crowd's response matched the gold standard increased with increasing agreement.

Based on these observations, the following normalized confidence score is proposed:
1. A normalized confidence score is a real number in the range [0, 1].
2. The normalized confidence score represents the accuracy-weighted percentage of the workers who chose the positive choice for a binary judgement. A score of 1.0 means 100% of workers chose the positive choice, and a score of 0.0 means 0% of the workers chose the positive choice (everyone chose the negative choice).
3. Worker performance is normalized based on accuracy for a set of test questions. This is the *trust* score for an individual worker.

The normalized confidence score *C<sub>i</sub>* for the CID relation *i* is defined as:
## $C_i = \frac{\sum\limits_{j}^p T_j}{\sum\limits_{k}^i T_k}, p \subseteq i$

where *p* represents the set of workers who chose the positive choice for the work unit, *w* represents all the workers who worked on the work unit, and *T* the trust score for each individual worker.

Experiments using majority voting for aggregating results showed that the exact same ROC curves could be produced if the confidence scoring was assigned as follows:

- Confidence score ranges from -1.0 to 1.0. Negative scores means that a majority chose the negative choice.
- Confidence score represents the weighted percentage of the crowd that chose the top answer.

This definition produces scores that are in the range -1.0 to -0.5 and 0.5 to 1.0 by definition of majority voting for a binary judgement. Since this is more complicated and produces the same results as the preceeding definition, the definition given above will be used henceforth.

### Aggregating Sentence-level Results

A single chemical-disease identifier pair may co-occur in multiple sentences within an abstract. Since each of the sentences becomes one CrowdFlower work unit, how do we aggregate the results together to produce a judgement about whether the relationship is true at the abstract level?

Since a vote of "no" for a relationship only means that the sentence itself does not contain enough information to support a true relationship, we assume that for a set of sentences containing the relationship, the relationship is true at the abstract level if at least one of the sentences contains the relationship. The confidence score for the relationship at the abstract level is the maximum of the confidence scores of the individual sentences.